# Distance and Nearest Neighbor Calculations
### by [Kate Vavra-Musser](https://vavramusser.github.io) for the [R Spatial Notebook Series](https://vavramusser.github.io/r-spatial)

## Introduction

Proximity analysis is a critical aspect of spatial analysis, enabling researchers to evaluate distances between spatial features, assess accessibility, and identify relationships between locations.  Proximity analysis is widely used in accessibility studies, urban planning, environmental research, and many other fields.

For this notebook, we’ll use the [*rnaturalearth*](https://cran.r-project.org/web/packages/rnaturalearth/index.html) package to access and load datasets from [*Natural Earth*](https://www.naturalearthdata.com). These packages provide direct access to Natural Earth’s geographic data without requiring an API key, making it simple to bring boundaries and point data directly into R.

This notebook introduces three commonly used geometric query functions from the [**`sf`**](https://cran.r-project.org/web/packages/sf/index.html) package:

* *`st_distance`*
* *`st_nearest_feature`*

### Notebook Goals
This notebook demonstrates how to perform proximity analysis using R, covering common scenarios and applications.

### ★ Prerequisites ★

#### About the Example Data Set

#### Notebook Overview
1. Setup
2. ...

## 1. Setup
This section will guide you through the process of installing essential packages and setting your IPUMS API key.

##### Required Packages

[**sf**](https://cran.r-project.org/web/packages/sf/index.html) · Support for simple features, a standardized way to encode spatial vector data. Binds to 'GDAL' for reading and writing data, to 'GEOS' for geometrical operations, and to 'PROJ' for projection conversions and datum transformations. Uses by default the 's2' package for spherical geometry operations on ellipsoidal (long/lat) coordinates.  This notebook uses the following functions from *sf*.

* [*geos_measures*](https://rdrr.io/cran/sf/man/geos_measures.html) · compute geometric measurements
  * *st_area* · compute area
  * *st_distance* · compute distance
  * *st_length* · compute length
  * *st_perimeter* · compute perimeter
* [*st_crs*](https://rdrr.io/cran/sf/man/st_crs.html) · retrieve coordinate reference system from object
* [*st_nearest_feature*](https://rdrr.io/cran/sf/man/st_nearest_feature.html) · get index of nearest featur
* [*st_read*](https://rdrr.io/cran/sf/man/st_read.html) · read simple features or layers from file or database
* [*st_transform*](https://rdrr.io/cran/sf/man/st_transform.html) · transform or convert coordinates of simple feature
* [*valid*](https://rdrr.io/cran/sf/man/valid.html) · check validity or make an invalid geometry valid
  * *st_is_valid* · check validity

### 1a. Install and Load Required Packages
If you have not already installed the required packages, uncomment and run the code below:

In [3]:
# install.packages("sf")

Load the packages into your workspace.

In [4]:
library(sf)

### 1b. Import Data

First we will import three vector datasets to work with using the [*st_read*](https://rdrr.io/cran/sf/man/st_read.html) function from the [**sf**](https://cran.r-project.org/web/packages/sf/index.html) package.  If you worked through [Chapter 3.4.1](), [Chapter 3.4.2](), and [Chapter 4.1](), you should have saved these files to your workspace and they should be available to work with directly.  If you did not work through these chapters, you can download and import the files to your workspace.

* **United States Populated Places Point Locations** (*ipums_nhgis_places.shp*) downloaded from [IPUMS NHGIS](https://www.nhgis.org) in [Chapter 3.4.1 IPUMS NHGIS Data Extraction Using ipumsr - Supplemental Exercise 1](). 
* **United States Country Boundary** (*usa_boundary.shp*) downloaded from [Natural Earth](https://www.naturalearthdata.com) in [Chapter 4.1 Importing Data from Natural Earth with rnaturalearth and rnaturalearthdata](). 
* **Global Airport Point Locations** (*airports.shp*) downloaded from [Natural Earth](https://www.naturalearthdata.com) in [Chapter 4.1 Importing Data from Natural Earth with rnaturalearth and rnaturalearthdata](). 

In [7]:
# Read sample data: Census tracts and public service locations
places <- st_read("ipums_nhgis_places.shp")
usa <- st_read("usa_boundary.shp")
airports <- st_read("airports.shp")

Reading layer `ipums_nhgis_places' from data source 
  `C:\Users\vavra\Dropbox\R Spatial\r-spatial\ipums_nhgis_places.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 29261 features and 22 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -6238595 ymin: -1328241 xmax: 2254129 ymax: 4577031
Projected CRS: USA_Contiguous_Albers_Equal_Area_Conic
Reading layer `usa_boundary' from data source 
  `C:\Users\vavra\Dropbox\R Spatial\r-spatial\usa_boundary.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 166 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -171.7911 ymin: 18.91619 xmax: -66.96466 ymax: 71.35776
Geodetic CRS:  WGS 84
Reading layer `airports' from data source 
  `C:\Users\vavra\Dropbox\R Spatial\r-spatial\airports.shp' using driver `ESRI Shapefile'
Simple feature collection with 893 features and 39 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: -175.1356 ymin: -53.78

We can see that the *places* file contains point locations and information on 29,261 populated places in the US, the *usa* file contains one boundary for the United States, and the *airports* file contains point locations and information on 893 major airports globally.

### 1c. Project All Files to the Same CRS

Let's ensure that all three files are using the same coordinate reference system (CRS).  Based on the file information output from reading in the data in the above code chunk, we can see that the *places* files from [IPUMS NHGIS](https://www.nhgis.org) uses the [USA Contiguous Albers Equal Area Conic projected CRS](https://epsg.io/102003) while the two files from [Natural Earth](https://www.naturalearthdata.com) (*usa* and *airports*) uses the [WGS 1984 Geodetic CRS](https://epsg.io/4326).

For this notebook, we will use the [WGS 1984 Geodetic CRS](https://epsg.io/4326) used by *usa* and *airports* so we will need to reproject the *places* files to this CRS.

In [8]:
# make versons
places <- st_transform(places, st_crs(usa))

Let's double check the CRS for all three files.

In [9]:
st_crs(places)$epsg
st_crs(usa)$epsg
st_crs(airports)$epsg

[1] 4326

[1] 4326

[1] 4326

All three files are now represented in the [WGS 1984 Geodetic CRS (EPSG code 4326)](https://epsg.io/4326).

### 1d. Validate the Data

Let's ensure that the three files we are working with have valid geometries using the [*st_is_valid*](https://rdrr.io/cran/sf/man/valid.html) function from the [**sf**](https://cran.r-project.org/web/packages/sf/index.html) package.  If there are any invalid geometries that could impact our ability to carry out spatial analyses or properly map our data.

In [10]:
table(st_is_valid(places))


 TRUE 
29261 

In [11]:
table(st_is_valid(usa))


TRUE 
   1 

In [12]:
table(st_is_valid(airports))


TRUE 
 893 

All 29,261 features in the *places* file, 1 feature in the *usa* file, and 893 features in the *airports* file are all valid!  We are ready to move on to the next step.

### 1e. Review the Data

Let's take a look at the first few rows for the *places* and *airports* files as a refresher to see what attibutes (columns) each file includes.

In [13]:
head(places)

Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



GISJOIN   GEOGYEAR STATE_x STATEA PLACE_x         PLACEA CL8AA1990 CL8AA1990L
1 G01000100 2010     Alabama 01     Abanda CDP      00100    177.25    120     
2 G01000124 2010     Alabama 01     Abbeville city  00124   3176.41   2996     
3 G01000460 2010     Alabama 01     Adamsville city 00460   5620.12   2558     
4 G01000484 2010     Alabama 01     Addison town    00484    715.01    278     
5 G01000676 2010     Alabama 01     Akron town      00676    444.66    284     
6 G01000820 2010     Alabama 01     Alabaster city  00820  15885.55  11096     
  CL8AA1990U CL8AA2000 geometry                   ⋯ CL8AA2020 CL8AA2020L
1   213        131.68  POINT (-85.52968 33.10095) ⋯   133.00    133     
2  3186       2987.18  POINT (-85.25049 31.57184) ⋯  2358.00   2358     
3 10483       5107.32  POINT (-86.95611 33.60094) ⋯  4353.93   2274     
4  1147        743.68  POINT (-87.1814 34.20232)  ⋯   659.00    659     
5   575        448.09  POINT (-87.74251 32.87652) ⋯   225.00    225     
6 19519      24415.70  POINT (-86.81638 33.24428) ⋯ 32519.23  25856     
  CL8AA2020U STATE_y NHGISST PLACE_y         NAME       NHGISPLACE YEAR
1   133      Alabama 010     Abanda CDP      Abanda     G010001000 2010
2  2358      Alabama 010     Abbeville city  Abbeville  G010001240 2010
3  6083      Alabama 010     Adamsville city Adamsville G010004600 2010
4   659      Alabama 010     Addison town    Addison    G010004840 2010
5   225      Alabama 010     Akron town      Akron      G010006760 2010
6 34631      Alabama 010     Alabaster city  Alabaster  G010008200 2010
  geometry                  
1 POINT (-85.52968 33.10095)
2 POINT (-85.25049 31.57184)
3 POINT (-86.95611 33.60094)
4 POINT (-87.1814 34.20232) 
5 POINT (-87.74251 32.87652)
6 POINT (-86.81638 33.24428)

In [14]:
head(airports)

scalerank featurecla type             name           abbrev location gps_code
1 9         Airport    small            Sahnewal       LUH    terminal VILD    
2 9         Airport    mid              Solapur        SSE    terminal VASL    
3 9         Airport    mid              Birsa Munda    IXR    terminal VERC    
4 9         Airport    mid              Ahwaz          AWZ    terminal OIAW    
5 9         Airport    mid and military Gwalior        GWL    terminal VIGR    
6 9         Airport    mid              Hodeidah Int'l HOD    terminal OYHD    
  iata_code wikipedia                                                 
1 LUH       http://en.wikipedia.org/wiki/Sahnewal_Airport             
2 SSE       http://en.wikipedia.org/wiki/Solapur_Airport              
3 IXR       http://en.wikipedia.org/wiki/Birsa_Munda_Airport          
4 AWZ       http://en.wikipedia.org/wiki/Ahwaz_Airport                
5 GWL       http://en.wikipedia.org/wiki/Gwalior_Airport              
6 HOD       http://en.wikipedia.org/wiki/Hodeida_International_Airport
  natlscale geometry                  ⋯ name_tr              
1 8         POINT (75.95707 30.85036) ⋯ NA                   
2 8         POINT (75.93306 17.62542) ⋯ NA                   
3 8         POINT (85.3236 23.31772)  ⋯ Birsa Munda Havaalan?
4 8         POINT (48.74711 31.34316) ⋯ Ahvaz Havaliman?     
5 8         POINT (78.21722 26.28549) ⋯ NA                   
6 8         POINT (42.9711 14.75525)  ⋯ NA                   
  name_vi             name_zh  wdid_score name_fa                  name_he
1 NA                  NA       4          ??????? ?????            NA     
2 NA                  NA       4          ??????? ???????          NA     
3 Sân bay Birsa Munda ???·???? 4          ??????? ????? ?????      NA     
4 Sân bay Ahvaz       ?????    4          ??????? ??? ?????? ????? NA     
5 Sân bay Gwalior     ??????   4          ??????? ???????          NA     
6 NA                  ???????  4          ??????? ????????? ?????? NA     
  name_uk name_ur name_zht geometry                 
1 NA      NA      NA       POINT (75.95707 30.85036)
2 NA      NA      NA       POINT (75.93306 17.62542)
3 NA      NA      ???·???? POINT (85.3236 23.31772) 
4 NA      NA      ?????    POINT (48.74711 31.34316)
5 NA      NA      ??????   POINT (78.21722 26.28549)
6 NA      NA      ???????  POINT (42.9711 14.75525)

Let's take a look at the different categories of the *type* and *location* variables in the *airports* file.

In [17]:
unique(airports$type)

[1] "small"              "mid"                "mid and military"  
[4] "major and military" "military mid"       "military"          
[7] "major"              "military major"     "spaceport"

In [18]:
unique(airports$location)

[1] "terminal"    "ramp"        "runway"      "approximate" "parking"    
[6] "freight"

A few attributes we may want to take note of for our analyses are:

From the ***places*** file:

* State Name (*STATE_x*)
* City Name (*PLACE_x*)
* 1990 Total Population (*CL8AA1990*)
* 2000 Total Population (*CL8AA2000*)
* 2010 Total Population (*CL8AA2010*)
* 2020 Total Population (*CL8AA2020*)

From the ***airports*** file:

* Airport Name (*name*)
* Airport Code (*abbrev*)
* Airport Type (*type*)
  * categories: small, mid, major, military, military mid, military major, mid and military, major and military, spaceport
* Airport Location (*location*)
  * categories: terminal, ramp, runway, approximate, parking, freight

## 2. Basic Distance Calculations

* Euclidean Distance (Planar)
  * Best for small-scale, local projections where curvature is negligible.
  * Used for calculating straight-line distances in projected coordinate systems.
* Great Circle Distance (Haversine)
  * Appropriate for geographic (longitude/latitude) coordinates.
  * Accounts for Earth's curvature using the Haversine formula.
* Manhattan Distance (Taxicab)
  * Measures distances in a grid-like structure, useful for urban environments.
* Minkowski Distance
  * Generalization of Euclidean and Manhattan distances; customizable via a parameter.
* Chebyshev Distance
  * Measures the greatest difference along any coordinate dimension.
  * Useful in chessboard-like movement patterns.

### 2a. Euclidean Distance (Planar)

### 2b. Great Circle Distance (Haversine)

### 2c. Manhattan Distance (Taxicab)

### 2d. Minkowski Distance

### 2e. Chebyshev Distance

In [39]:
# New York City
nyc <- places[places$PLACE_x == "New York city",]

# John F Kennedy Airport
jfk <- airports[581,]

character; for Cartesian coordinates only: one of Euclidean, Hausdorff or
Frech

In [40]:
st_distance(nyc, jfk)

Units: [m]
         [,1]
[1,] 20018.84

The distance between the point location representing New York City and the point location representing John F Kennedy Airport is approximately 20,019 meters or about 12.4 miles.

### 3a. Euclidean Distance

Euclidean distance is the "straight-line" distance between two points in Cartesian space. It is calculated based on projected coordinates and is often used for smaller areas.

In [16]:
# find the nearest airport for each place
nearest_airport_indices <- st_nearest_feature(usa_places, usa_airports)

In [17]:
# calculate the distance to the nearest airport
distances <- st_distance(usa_places, usa_airports[nearest_airport_indices, ], by_element = T)

In [18]:
head(distances)

Units: [m]
[1] 118526.92 134800.41  19281.30  62137.04 119424.51  35953.89

### 3b. Geodesic Distance

Geodesic distance is the shortest path between two points on the Earth's surface. It is more accurate for larger areas or when working with lat/lon coordinates.

In [ ]:
### requires the geosphere package ###

# Calculate geodesic distances using geosphere package
#coords <- st_coordinates(cities)
#dist_matrix_geodesic <- distm(
#  x = coords[1:10, ],
#  fun = distHaversine  # Haversine formula for geodesic distance
#)

# Display the distance matrix (in meters)
#dist_matrix_geodesic

### 3b. Distance Matrix

In [20]:
# Transform to projected CRS for accurate Euclidean distance
#cities_projected <- st_transform(cities_within_us, crs = 26915)  # UTM Zone 15N

# Select a sample of cities for analysis
#cities_projected <- cities_projected[1:10, ]

# Distance matrix (Euclidean)
#usa_airports_dist_matrix_euclidean <- st_distance(usa_airports)

# Display the distance matrix (in meters)
#usa_airports_dist_matrix_euclidean

Based this analysis, San Juan, Puerto Rico is the closest major city to the coast while Denver, Colorado is the furthest major city from the coast.

## 4. Nearest Neighbor Analysis

Nearest neighbor analysis identifies the closest spatial feature for each point in a dataset. This is useful for applications like finding the nearest service center or analyzing spatial clustering.

In [ ]:
# Find the index of the nearest coastline feature for each city
nearest_indices <- st_nearest_feature(cities, coastline)

# Extract the corresponding distances
nearest_distances <- st_distance(cities, coastline[nearest_indices, ])

# Convert distances to kilometers
nearest_distances_km <- as.numeric(nearest_distances) / 1000

# Combine results into a data frame
nearest_results <- data.frame(
  city_name = cities$NAME,
  nearest_coastline_index = nearest_indices,
  distance_to_coastline_km = nearest_distances_km
)

# Display nearest neighbor results
nearest_results

## 5. Buffer Analysis

Buffer analysis creates zones of influence around spatial features, which can be used to analyze proximity impacts, such as areas within a certain distance of cities.

# Create buffer zones around cities
buffers <- st_buffer(cities_projected[1:10, ], dist = 50000)  # 50 km buffer

# Check which coastlines intersect with buffers
intersections <- st_intersects(buffers, coastline)

# Summarize results
buffer_results <- data.frame(
  city_name = cities$NAME[1:10],
  num_coastline_intersections = sapply(intersections, length)
)

# Display buffer analysis results
buffer_results

# Visualize buffers on a map
leaflet() %>%
  addTiles() %>%
  addPolygons(data = st_transform(buffers, crs = 4326), color = "green", fillOpacity = 0.2, group = "Buffers") %>%
  addCircleMarkers(data = cities, color = "blue", radius = 5, group = "Cities") %>%
  addPolylines(data = coastline, color = "red", group = "Coastline") %>%
  addLayersControl(overlayGroups = c("Buffers", "Cities", "Coastline"))